# Ruby Kaigi 2016, Kyoto

## Statistical analysis with daru and statsample-glm

## Data stored in daru data structures can be used with statistical methods in statsample-glm

### In this demo we perform a logistic regression with data taken from a Kaggle data set about the putcome of animals from animal shelters in Austin, Texas.

In [1]:
require 'daru'

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

true

### Let's load the training data set from a CSV file.

In [2]:
shelter_data = Daru::DataFrame.from_csv 'animal_shelter_train.csv'

#<Daru::DataFrame(26729x10)>
              AnimalID       Name   DateTime OutcomeTyp OutcomeSub AnimalType SexuponOut AgeuponOut      Breed      Color
          0    A671945    Hambone 2014-02-12 Return_to_        nil        Dog Neutered M     1 year Shetland S Brown/Whit
          1    A656520      Emily 2013-10-13 Euthanasia  Suffering        Cat Spayed Fem     1 year Domestic S Cream Tabb
          2    A686464     Pearce 2015-01-31   Adoption     Foster        Dog Neutered M    2 years Pit Bull M Blue/White
          3    A683430        nil 2014-07-11   Transfer    Partner        Cat Intact Mal    3 weeks Domestic S Blue Cream
          4    A667013        nil 2013-11-15   Transfer    Partner        Dog Neutered M    2 years Lhasa Apso        Tan
          5    A677334       Elsa 2014-04-25   Transfer    Partner        Dog Intact Fem    1 month Cairn Terr  Black/Tan
          6    A699218      Jimmy 2015-03-28   Transfer    Partner        Cat Intact Mal    3 weeks Domestic S Blue Tabby
          7    A701489        nil 2015-04-30   Transfer    Partner        Cat    Unknown    3 weeks Domestic S Brown Tabb
          8    A671784       Lucy 2014-02-04   Adoption        nil        Dog Spayed Fem   5 months American P  Red/White
          9    A677747        nil 2014-05-03   Adoption    Offsite        Dog Spayed Fem     1 year Cairn Terr      White
         10    A668402        nil 2013-12-05   Transfer       SCRP        Cat    Unknown    2 years Domestic S      Black
         11    A666320        nil 2013-11-04   Adoption        nil        Dog Spayed Fem    2 years Miniature      Silver
         12    A684601     Rocket 2016-02-03   Adoption     Foster        Dog Neutered M    4 years Pit Bull M      Brown
         13    A704702    Scooter 2015-06-08 Return_to_        nil        Dog Neutered M    2 years Yorkshire   Black/Red
         14    A688584    Preston 2015-11-25 Return_to_        nil        Dog Neutered M     1 year Great Pyre White/Crea
        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...        ...

# Goal : Predict the OutcomeType of each animal in the test data.

### Let's see the unique outcome types with `Vector#uniq`.

In [3]:
shelter_data["OutcomeType"].uniq

#<Daru::Vector(5)>
                     OutcomeType
               0 Return_to_owner
               1      Euthanasia
               2        Adoption
               3        Transfer
             553            Died

## Pre-processing the training data

### For simplicity, we keep only the `"AgeuponOutcome"`,`"AnimalType"`, `"Breed"`, `"Color"`, and `"SexuponOutcome"` as the predictor variables for determining the `"OutcomeType"`.

In [4]:
shelter_data.delete_vectors *%W[AnimalID Name DateTime OutcomeSubtype]

#<Daru::DataFrame(26729x6)>
            OutcomeTyp AnimalType SexuponOut AgeuponOut      Breed      Color
          0 Return_to_        Dog Neutered M     1 year Shetland S Brown/Whit
          1 Euthanasia        Cat Spayed Fem     1 year Domestic S Cream Tabb
          2   Adoption        Dog Neutered M    2 years Pit Bull M Blue/White
          3   Transfer        Cat Intact Mal    3 weeks Domestic S Blue Cream
          4   Transfer        Dog Neutered M    2 years Lhasa Apso        Tan
          5   Transfer        Dog Intact Fem    1 month Cairn Terr  Black/Tan
          6   Transfer        Cat Intact Mal    3 weeks Domestic S Blue Tabby
          7   Transfer        Cat    Unknown    3 weeks Domestic S Brown Tabb
          8   Adoption        Dog Spayed Fem   5 months American P  Red/White
          9   Adoption        Dog Spayed Fem     1 year Cairn Terr      White
         10   Transfer        Cat    Unknown    2 years Domestic S      Black
         11   Adoption        Dog Spayed Fem    2 years Miniature      Silver
         12   Adoption        Dog Neutered M    4 years Pit Bull M      Brown
         13 Return_to_        Dog Neutered M    2 years Yorkshire   Black/Red
         14 Return_to_        Dog Neutered M     1 year Great Pyre White/Crea
        ...        ...        ...        ...        ...        ...        ...

## We also remove all the rows of the dataframe that contain any missing data.

In [5]:
shelter_data = shelter_data.filter_rows { |row| !row.include_values? *Daru::MISSING_VALUES }

#<Daru::DataFrame(26710x6)>
            OutcomeTyp AnimalType SexuponOut AgeuponOut      Breed      Color
          0 Return_to_        Dog Neutered M     1 year Shetland S Brown/Whit
          1 Euthanasia        Cat Spayed Fem     1 year Domestic S Cream Tabb
          2   Adoption        Dog Neutered M    2 years Pit Bull M Blue/White
          3   Transfer        Cat Intact Mal    3 weeks Domestic S Blue Cream
          4   Transfer        Dog Neutered M    2 years Lhasa Apso        Tan
          5   Transfer        Dog Intact Fem    1 month Cairn Terr  Black/Tan
          6   Transfer        Cat Intact Mal    3 weeks Domestic S Blue Tabby
          7   Transfer        Cat    Unknown    3 weeks Domestic S Brown Tabb
          8   Adoption        Dog Spayed Fem   5 months American P  Red/White
          9   Adoption        Dog Spayed Fem     1 year Cairn Terr      White
         10   Transfer        Cat    Unknown    2 years Domestic S      Black
         11   Adoption        Dog Spayed Fem    2 years Miniature      Silver
         12   Adoption        Dog Neutered M    4 years Pit Bull M      Brown
         13 Return_to_        Dog Neutered M    2 years Yorkshire   Black/Red
         14 Return_to_        Dog Neutered M     1 year Great Pyre White/Crea
        ...        ...        ...        ...        ...        ...        ...

## Notice that `"AgeUponOutcome"` is given in different units. So let's ensure that it's all in the same units (weeks).

In [6]:
shelter_data['AgeuponOutcome'].map! do |age|
  num, unit = age.split
  num = num.to_f
  case unit
  when "year", "years"
    52.0 * num
  when "month", "months"
    4.5 * num
  when "week", "weeks"
    num
  when "day", "days"
    num / 7.0
  else
    raise "Unknown AgeuponOutcome unit!"
  end  
end
shelter_data.head

#<Daru::DataFrame(10x6)>
            OutcomeTyp AnimalType SexuponOut AgeuponOut      Breed      Color
          0 Return_to_        Dog Neutered M       52.0 Shetland S Brown/Whit
          1 Euthanasia        Cat Spayed Fem       52.0 Domestic S Cream Tabb
          2   Adoption        Dog Neutered M      104.0 Pit Bull M Blue/White
          3   Transfer        Cat Intact Mal        3.0 Domestic S Blue Cream
          4   Transfer        Dog Neutered M      104.0 Lhasa Apso        Tan
          5   Transfer        Dog Intact Fem        4.5 Cairn Terr  Black/Tan
          6   Transfer        Cat Intact Mal        3.0 Domestic S Blue Tabby
          7   Transfer        Cat    Unknown        3.0 Domestic S Brown Tabb
          8   Adoption        Dog Spayed Fem       22.5 American P  Red/White
          9   Adoption        Dog Spayed Fem       52.0 Cairn Terr      White

## Since this data involves categorical data, we must tell the DataFrame which variables are categorical in nature.

### This can be done with `DataFrame#to_category`.

In [7]:
shelter_data.to_category 'OutcomeType', 'AnimalType', 'SexuponOutcome', 'Breed', 'Color'

#<Daru::DataFrame(26710x6)>
            OutcomeTyp AnimalType SexuponOut AgeuponOut      Breed      Color
          0 Return_to_        Dog Neutered M       52.0 Shetland S Brown/Whit
          1 Euthanasia        Cat Spayed Fem       52.0 Domestic S Cream Tabb
          2   Adoption        Dog Neutered M      104.0 Pit Bull M Blue/White
          3   Transfer        Cat Intact Mal        3.0 Domestic S Blue Cream
          4   Transfer        Dog Neutered M      104.0 Lhasa Apso        Tan
          5   Transfer        Dog Intact Fem        4.5 Cairn Terr  Black/Tan
          6   Transfer        Cat Intact Mal        3.0 Domestic S Blue Tabby
          7   Transfer        Cat    Unknown        3.0 Domestic S Brown Tabb
          8   Adoption        Dog Spayed Fem       22.5 American P  Red/White
          9   Adoption        Dog Spayed Fem       52.0 Cairn Terr      White
         10   Transfer        Cat    Unknown      104.0 Domestic S      Black
         11   Adoption        Dog Spayed Fem      104.0 Miniature      Silver
         12   Adoption        Dog Neutered M      208.0 Pit Bull M      Brown
         13 Return_to_        Dog Neutered M      104.0 Yorkshire   Black/Red
         14 Return_to_        Dog Neutered M       52.0 Great Pyre White/Crea
        ...        ...        ...        ...        ...        ...        ...

## Now, the `"Breed"` variable has too many categories (almost 1000)!

In [8]:
shelter_data["Breed"].describe

#<Daru::Vector(6)>
                 size                26710
           categories                 1380
             max_freq                 8794
         max_category Domestic Shorthair M
             min_freq                    1
         min_category Beagle/Treeing Walke

## This might create complications when creating our model.

## So we reduce the number of breeds in the data by recoding rare breeds as 'other' (rare as in occuring less than 200 times).

In [9]:
# Create an Array of breeds that occur less than 100 times in the "Breed" vector
rare_breeds = shelter_data['Breed'].categories.select do |breed| 
  shelter_data['Breed'].count(breed) < 200 
end
shelter_data['Breed'].replace_values rare_breeds, 'other'
shelter_data['Breed'].base_category = 'other'
shelter_data['Breed'].describe

#<Daru::Vector(6)>
              size             26710
        categories                14
          max_freq              9167
      max_category             other
          min_freq               229
      min_category Border Collie Mix

## As can be seen above, we have now reduced the number of breed varieties to 28.

## There are also too many types of `"Color"`.

In [10]:
shelter_data["Color"].describe

#<Daru::Vector(6)>
          size         26710
    categories           366
      max_freq          2824
  max_category   Black/White
      min_freq             1
  min_category Tortie/Calico

## 366 colors! Nobody can distinguish those many anyway.
## So lets reduce the number of colors in the data set.

In [11]:
other_color = shelter_data['Color'].categories.select do |i| 
  shelter_data['Color'].count(i) < 200
end
shelter_data['Color'].replace_values other_color, 'other'
shelter_data['Color'].base_category = 'other'
shelter_data['Color'].describe

#<Daru::Vector(6)>
            size           26710
      categories              31
        max_freq            5307
    max_category           other
        min_freq             224
    min_category Chocolate/White

## There are now only 31 different colors!

## Let us now take the first 500 rows of the data frame as a training data set.

In [12]:
train_data = shelter_data.head 500

#<Daru::DataFrame(500x6)>
            OutcomeTyp AnimalType SexuponOut AgeuponOut      Breed      Color
          0 Return_to_        Dog Neutered M       52.0      other Brown/Whit
          1 Euthanasia        Cat Spayed Fem       52.0 Domestic S      other
          2   Adoption        Dog Neutered M      104.0 Pit Bull M Blue/White
          3   Transfer        Cat Intact Mal        3.0 Domestic S      other
          4   Transfer        Dog Neutered M      104.0      other        Tan
          5   Transfer        Dog Intact Fem        4.5      other  Black/Tan
          6   Transfer        Cat Intact Mal        3.0 Domestic S Blue Tabby
          7   Transfer        Cat    Unknown        3.0 Domestic S Brown Tabb
          8   Adoption        Dog Spayed Fem       22.5      other  Red/White
          9   Adoption        Dog Spayed Fem       52.0      other      White
         10   Transfer        Cat    Unknown      104.0 Domestic S      Black
         11   Adoption        Dog Spayed Fem      104.0      other      other
         12   Adoption        Dog Neutered M      208.0 Pit Bull M      Brown
         13 Return_to_        Dog Neutered M      104.0      other      other
         14 Return_to_        Dog Neutered M       52.0      other      other
        ...        ...        ...        ...        ...        ...        ...

## ... and 5 random rows from the data set as testing data.

In [13]:
test_data = shelter_data.row[754,653,9021,3000,5043]
test_data.delete_vectors 'OutcomeType'

#<Daru::DataFrame(5x5)>
            AnimalType SexuponOut AgeuponOut      Breed      Color
        754        Cat Intact Fem      104.0 Domestic S     Calico
        653        Cat Spayed Fem       52.0 Domestic S      White
       9021        Dog Intact Fem      312.0 Chihuahua       Brown
       3000        Dog Intact Mal      104.0 Miniature       White
       5043        Dog Intact Fem      260.0 Chihuahua         Tan

## We now define a dummy variable for the OutcomeType for predicting the outcome of "Adoption" using a coding scheme to define a 0-1 valued variable.

In [14]:
train_data["Adoption"] = (train_data["OutcomeType"].contrast_code)["OutcomeType_Adoption"]
train_data

#<Daru::DataFrame(500x7)>
            OutcomeTyp AnimalType SexuponOut AgeuponOut      Breed      Color   Adoption
          0 Return_to_        Dog Neutered M       52.0      other Brown/Whit          0
          1 Euthanasia        Cat Spayed Fem       52.0 Domestic S      other          0
          2   Adoption        Dog Neutered M      104.0 Pit Bull M Blue/White          1
          3   Transfer        Cat Intact Mal        3.0 Domestic S      other          0
          4   Transfer        Dog Neutered M      104.0      other        Tan          0
          5   Transfer        Dog Intact Fem        4.5      other  Black/Tan          0
          6   Transfer        Cat Intact Mal        3.0 Domestic S Blue Tabby          0
          7   Transfer        Cat    Unknown        3.0 Domestic S Brown Tabb          0
          8   Adoption        Dog Spayed Fem       22.5      other  Red/White          1
          9   Adoption        Dog Spayed Fem       52.0      other      White          1
         10   Transfer        Cat    Unknown      104.0 Domestic S      Black          0
         11   Adoption        Dog Spayed Fem      104.0      other      other          1
         12   Adoption        Dog Neutered M      208.0 Pit Bull M      Brown          1
         13 Return_to_        Dog Neutered M      104.0      other      other          0
         14 Return_to_        Dog Neutered M       52.0      other      other          0
        ...        ...        ...        ...        ...        ...        ...        ...

## We can now train a model to predict whether a pet will be adopted using statsample-glm

In [15]:
require 'statsample-glm'

true

## Statsample-glm supports a formula language like R or Patsy

### In the formula below, `Adoption` is the response variable and the variables to it's right are the predictor variables.

In [16]:
formula = 'Adoption ~ AnimalType + Breed + AgeuponOutcome + Color + SexuponOutcome'

"Adoption ~ AnimalType + Breed + AgeuponOutcome + Color + SexuponOutcome"

## We now fit a logistic regression on the training data set using the `Statsample::GLM::Regression` class.

In [17]:
glm_adoption = Statsample::GLM::Regression.new formula, train_data, :logistic

#<Statsample::GLM::Regression:0x9ace30c @formula=#<Statsample::GLM::FormulaWrapper:0x9ace258 @df=#<Daru::DataFrame(500x7)>
            OutcomeTyp AnimalType SexuponOut AgeuponOut      Breed      Color   Adoption
          0 Return_to_        Dog Neutered M       52.0      other Brown/Whit          0
          1 Euthanasia        Cat Spayed Fem       52.0 Domestic S      other          0
          2   Adoption        Dog Neutered M      104.0 Pit Bull M Blue/White          1
          3   Transfer        Cat Intact Mal        3.0 Domestic S      other          0
          4   Transfer        Dog Neutered M      104.0      other        Tan          0
          5   Transfer        Dog Intact Fem        4.5      other  Black/Tan          0
          6   Transfer        Cat Intact Mal        3.0 Domestic S Blue Tabby          0
          7   Transfer        Cat    Unknown        3.0 Domestic S Brown Tabb          0
          8   Adoption        Dog Spayed Fem       22.5      other  Red/Whit

## We can now calculate the logistic regression co-efficients of the test data set.

In [18]:
m = glm_adoption.model.coefficients :hash

{:AnimalType_Cat=>-0.29366812869946046, :"Breed_Pit Bull Mix"=>-0.9040231102398557, :"Breed_German Shepherd Mix"=>0.07567649669602779, :"Breed_Chihuahua Shorthair Mix"=>0.1636722701881535, :"Breed_Labrador Retriever Mix"=>-0.5608236450433959, :"Breed_Domestic Longhair Mix"=>1.9733895516331863, :"Breed_Dachshund Mix"=>0.04666999576176688, :"Breed_Siamese Mix"=>1.7827909952550098, :"Breed_Domestic Medium Hair Mix"=>-0.1813512795130199, :"Breed_Border Collie Mix"=>-1.0949121883083859, :"Breed_Boxer Mix"=>-0.38956375099391666, :"Breed_Australian Cattle Dog Mix"=>-0.6726175662938817, :"Breed_Miniature Poodle Mix"=>1.5962013413375362, :Breed_other=>-0.1979430214210797, :"Color_Blue/White"=>0.11213092812719921, :Color_Tan=>-0.2120800787678099, :"Color_Black/Tan"=>-2.9752697795787637, :"Color_Blue Tabby"=>0.3419414038420273, :"Color_Brown Tabby"=>0.7340836595016543, :"Color_Red/White"=>1.314312392268307, :Color_White=>-0.3455153402145956, :Color_Black=>0.5753950814348071, :Color_Brown=>-0.3447

## The `#predict` method can be used for directly calculating the probablility of adoption.

### This is the test data:

In [19]:
test_data

#<Daru::DataFrame(5x5)>
            AnimalType SexuponOut AgeuponOut      Breed      Color
        754        Cat Intact Fem      104.0 Domestic S     Calico
        653        Cat Spayed Fem       52.0 Domestic S      White
       9021        Dog Intact Fem      312.0 Chihuahua       Brown
       3000        Dog Intact Mal      104.0 Miniature       White
       5043        Dog Intact Fem      260.0 Chihuahua         Tan

### The probability of an animal being adopted can be computed as follows:

In [20]:
adoption_probability = glm_adoption.predict test_data
test_data['Probability of Adoption'] = adoption_probability.to_a
test_data

#<Daru::DataFrame(5x6)>
            AnimalType SexuponOut AgeuponOut      Breed      Color Probabilit
        754        Cat Intact Fem      104.0 Domestic S     Calico 0.02480401
        653        Cat Spayed Fem       52.0 Domestic S      White 0.65980598
       9021        Dog Intact Fem      312.0 Chihuahua       Brown 0.00736127
       3000        Dog Intact Mal      104.0 Miniature       White 0.10242722
       5043        Dog Intact Fem      260.0 Chihuahua         Tan 0.01231695